In [32]:
import cv2
import numpy as np

# Load the pre-trained car detection classifier
car_cascade = cv2.CascadeClassifier('haarCascadeCars.xml')

# Open the video file
#cap = cv2.VideoCapture('cars.mp4')
cap = cv2.VideoCapture('MWay_traffic.wmv')

# Define the colors for the boxes
colours = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255), (128, 0, 0), 
          (0, 128, 0), (0, 0, 128), (128, 128, 0), (128, 0, 128), (255, 128, 0), 
          (255, 0, 128), (128, 255, 0), (0, 255, 128), (128, 0, 255), (0, 128, 255)]

# Dictionary to keep track of cars and their IDs
car_dict = {}

# Define the threshold distance to match cars between frames
match_threshold = 50

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # If there are no more frames, break out of the loop
    if not ret:
        break
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect cars in the frame
    cars = car_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=2, minSize=(20, 20))
    
    # Dictionary to keep track of matched cars in the current frame
    matched_cars = {}
    
    # Draw bounding boxes around the detected cars
    for i, (x, y, w, h) in enumerate(cars):
        centroid = (x + w//2, y + h//2)
        colour = (255, 0, 0)  # Default color
        
        # Match current car with the closest previous car by centroid distance
        closest_car_id = None
        closest_distance = np.inf
        for car_id, car_info in car_dict.items():
            previous_centroid = car_info['centroid']
            distance = np.linalg.norm(np.array(centroid) - np.array(previous_centroid))
            if distance < closest_distance:
                closest_distance = distance
                closest_car_id = car_id
        
        # If the closest previous car is within the match threshold, assume it's the same car
        if closest_distance < match_threshold:
            matched_cars[closest_car_id] = centroid
            colour = car_dict[closest_car_id]['colour']
    
        # If not, assign a new ID and color to the car
        else:
            new_car_id = max(car_dict.keys()) + 1 if car_dict else 0
            matched_cars[new_car_id] = centroid
            colour = colours[new_car_id % len(colours)]
        
        # Draw the bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), colour, 2)
        cv2.putText(frame, f'Car {closest_car_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour, 2)
        
            # Update the car's color in the dictionary
    if closest_car_id in car_dict:
        car_dict[closest_car_id]['colour'] = colour
    
    # If the car was not matched in the current frame, remove it from the dictionary
    remove_keys = []
    for car_id in car_dict.keys():
        if car_id not in matched_cars.keys():
            remove_keys.append(car_id)
    for car_id in remove_keys:
        car_dict.pop(car_id)
        
    # Add new cars to the dictionary
    for car_id, centroid in matched_cars.items():
        if car_id not in car_dict.keys():
            car_dict[car_id] = {'centroid': centroid, 'colour': colour}
            
    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Exit if the user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
